In [2]:
from cpr.Nivel import Nivel
from cpr.SqlDb import SqlDb
import cpr.information as info
import numpy as np
import os
import pandas as pd
import datetime

def duplicate_existing_table(self,table_name):
    '''
    inserts data into SQL table from list of fields and values
    Parameters
    ----------
    table_name   = SQL db table name
    Returns
    -------
    Sql sentence,str
    '''
    df = self.read_sql('describe %s'%table_name)
    df['Null'][df['Null']=='NO'] = 'NOT NULL'
    df['Null'][df['Null']=='YES'] = 'NULL'
    sentence = 'CREATE TABLE %s ('%table_name
    if df[df['Extra']=='auto_increment'].empty:
        pk = None
    else:
        pk = df[df['Extra']=='auto_increment']['Field'].values[0]
    for id,serie in df.iterrows():
        if (serie.Default=='0') or (serie.Default is None):
            row = '%s %s %s'%(serie.Field,serie.Type,serie.Null)
        else:
            if (serie.Default == 'CURRENT_TIMESTAMP'):
                serie.Default = "DEFAULT %s"%serie.Default
            elif serie.Default == '0000-00-00':
                serie.Default = "DEFAULT '1000-01-01 00:00:00'"
            else:
                serie.Default = "DEFAULT '%s'"%serie.Default
            row = '%s %s %s %s'%(serie.Field,serie.Type,serie.Null,serie.Default)
        if serie.Extra:
            row += ' %s,'%serie.Extra
        else:
            row += ',' 
        sentence+=row
    if pk:
        sentence +='PRIMARY KEY (%s)'%pk
    else:
        sentence = sentence[:-1]
    sentence +=');'
    return sentence

def siata_remote_data_to_transfer(start,end):
    '''
    inserts data into SQL table from list of fields and values
    Parameters
    ----------
    table_name   = SQL db table name
    Returns
    -------
    Sql sentence,str
    '''
    remote = cpr.Nivel(**cpr.info.REMOTE)
    codigos_str = '('+str(list(self.infost.index)).strip('[]')+')'
    df = remote.read_sql('SELECT * FROM datos WHERE cliente in %s and %s'%(codigos_str,self.fecha_hora_query(start,end)))
    return df

def default_values(self,table_name):
    '''
    inserts data into SQL table from list of fields and values
    Parameters
    ----------
    table_name   = SQL db table name
    Returns
    -------
    Sql sentence,str
    '''
    describe_table = self.read_sql('describe %s'%table_name)
    not_null = describe_table['Default'].notnull()
    default_values = describe_table[['Field','Default','Type']][not_null].set_index('Field')[['Default','Type']]
    return default_values

def filter_data_to_update(table_name,path):
    '''
    inserts data into SQL table from list of fields and values
    Parameters
    ----------
    table_name   = SQL db table name
    Returns
    -------
    Sql sentence,str
    '''
    default = default_values(self,table_name)
    if default[default['Type']=='time'].empty:
        time_fields = None
    else:
        time_fields = default[default['Type']=='time']
    for index,s in time_fields.iterrows():
        df = pd.read_csv(path,index_col=0)
        df[index] = df[index].apply(lambda x:str(x)[6:15])
    return df.applymap(lambda x:str(x))

def insert_in_datos(path):
    '''
    inserts data into SQL table from list of fields and values
    Parameters
    ----------
    table_name   = SQL db table name
    Returns
    -------
    Sql sentence,str
    '''
    inicia = datetime.datetime.now()
    table_name = 'datos'
    df = filter_data_to_update(table_name,path)
    query = 'INSERT INTO %s '%table_name+'('+str(list(df.columns)).strip('[]').replace("'",'')+') VALUES '
    for id,s in df.iterrows():
        query+=('('+str(list(s.values)).strip('[]'))+'), '
    query = query[:-2]+' ON DUPLICATE KEY UPDATE '
    describe = self.read_sql('describe %s;'%table_name)
    not_primary_keys = describe[describe['Key']!='PRI']
    if not_primary_keys.empty:
        not_primary_keys = describe.Field.values
    else:
        not_primary_keys = not_primary_keys.Field.values
    for key in not_primary_keys:
        query+=('%s = VALUES(%s), '%(key,key))
    query=query[:-2]
    finaliza = datetime.datetime.now()
    self.user = 'root'
    self.passwd = 'mcanoYw2E#'
    self.execute_sql('SET GLOBAL max_allowed_packet=1073741824;')
    self.execute_sql(query)
    print(finaliza - inicia)
    
#os.system("scp -r mcano@siata.gov.co:data_migration/* ../../cprweb/src/media/data_migration/")

/usr/local/lib/python2.7/dist-packages/wmf/wmf.py:18: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python2.7/dist-packages/tornado

recordar modificar info.DATA_PATH

In [5]:
info.DATA_PATH

'/home/mcano/Dev/cprweb/src/media/'

In [ ]:
self = SqlDb(**info.REMOTE)
folder_path = info.DATA_PATH+'weekly_data'
files = os.listdir(folder_path)

for path in files:
    insert_in_datos(folder_path+path)
    print(path)
level = cpr.Nivel(codigo=99,**cpr.info.LOCAL).level('2018-03-01',datetime.datetime.now())

april = [] # para agregar abril a nivel futuro
for path in files:
    if path[:20] == 'weekly_level_2018-04':
        april.append(path)

date_offset = pd.to_datetime(level.dropna().index[-1])
date = date_offset-pd.to_datetime(pd.read_csv(folder_path + april[0],index_col=0)['fecha'][0])
for path in april:
    raw_data = pd.read_csv(folder_path + path,index_col=0)
    raw_data['fecha'] = (pd.to_datetime(raw_data['fecha']) + datetime.timedelta(days=date.days)).apply(lambda x:x.strftime('%Y-%m-%d'))
    raw_data.to_csv(folder_path+'future_'+path)
    insert_in_datos(folder_path+'future_'+path)

# aforos migration

ALTER TABLE myusers_hydrodata ADD UNIQUE (fk_id,fecha);

In [5]:
self = Nivel(codigo=99,**info.LOCAL)
aforo_nueva = pd.read_csv('/home/mcano/Dev/cprweb/src/media/data_migration/aforo_nueva.csv').set_index('id_aforo')[['fecha','id_estacion_asociada']]
levantamiento_aforo_nueva = pd.read_csv('/home/mcano/Dev/cprweb/src/media/data_migration/levantamiento_aforo_nueva.csv')
levantamiento_aforo_nueva = levantamiento_aforo_nueva[['id_aforo','vertical','x','y']].set_index('id_aforo')
df = aforo_nueva.join(levantamiento_aforo_nueva)

In [7]:
ids = list(set(df.index))
aforo_nueva = aforo_nueva.loc[ids]

indexes = self.read_sql('select id,codigo from estaciones_estaciones').set_index('codigo')['id']
indexes.index = np.array(indexes.index,int)
date_format = '%Y-%m-%d %H:%M'
table = 'myusers_item'
now = datetime.datetime.now().strftime(date_format)

revisar = []
query = 'INSERT INTO %s (fecha,hora,minuto,timestamp,updated,offset,item_fk_id,user_id,tipo_salida,x_lamina) VALUES '%(table)

for id_aforo,s in aforo_nueva.iterrows():
    count = 0
    try:
        codigo = s.id_estacion_asociada
        fecha = pd.to_datetime(s.fecha)
        hora = int(fecha.strftime('%H'))
        item_fk_id = indexes[codigo]
        offset = self.infost.loc[codigo,'offset']
        x_lamina = self.infost.loc[codigo,'x_sensor']
        try:
            id = self.read_sql("select id from myusers_item where item_fk_id = '%s' and fecha = '%s' and hora = '%s'"%(item_fk_id,fecha.strftime('%Y-%m-%d'),hora)).iloc[0]['id']
        except:
            id = None
        if id:
            pass
        else:
            format = (fecha.strftime('%Y-%m-%d'),
                      str(hora),
                      str(int(fecha.strftime('%M'))),
                      now,
                      now,
                      offset,
                      str(item_fk_id),
                      str(1),
                      'siata-batimetria',
                     str(x_lamina))
            query += str(format)+','
            count += 1
    except:
        revisar.append(id_aforo)
query = query[:-1]
if count == 0:
    query = None
if query:
    self.execute_sql(query)

In [6]:
self = Nivel(codigo=99,**info.LOCAL)
table = 'myusers_topo'
query = 'INSERT INTO %s (user_id,fk_id,vertical,x,y,timestamp,updated) VALUES '%(table)
date_format = '%Y-%m-%d %H:%M'
now = datetime.datetime.now().strftime(date_format)
indexes = self.read_sql('select id,codigo from estaciones_estaciones').set_index('codigo')['id']
indexes.index = np.array(indexes.index,int)

for id_aforo,s in aforo_nueva.iterrows():
    try:
        count = 0
        codigo = s.id_estacion_asociada
        fecha = pd.to_datetime(s.fecha)
        hora = int(fecha.strftime('%H'))
        item_fk_id = indexes[codigo]
        x_lamina = float(self.infost.loc[codigo,'x_sensor'])
        id = self.read_sql("select id from myusers_item where item_fk_id = '%s' and fecha = '%s' and hora = '%s'"%(item_fk_id,fecha.strftime('%Y-%m-%d'),hora)).iloc[0]['id']
        lev = levantamiento_aforo_nueva.loc[id_aforo]
        lev = self.adjust_topo(lev,x_lamina).reset_index()
        lev.columns = ['vertical','x','y']
        for loc in range(lev.index.size):
            query +=  str((str(1),
                       str(int(id)),
                       str(int(lev.iloc[loc].vertical)),
                       str(lev.iloc[loc].x),
                       str(lev.iloc[loc].y),
                       now,
                       now))+','
    except:
        pass
query = query[:-1]
self.execute_sql(query)